# Assignment 6

In [65]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [66]:
# Dataset is already loaded below
data = pd.read_csv("employee.csv")

In [67]:
# Dimensions of training data
data.shape

(4277, 13)

In [68]:
# Print first few rows of data
data.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [69]:
# drop id, timestamp and country columns
data = data.drop(columns=['id', 'timestamp','country'])

In [70]:
# Explore columns
data.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [71]:
# replace NANs in hours_per_week with median value of the column
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()

In [72]:
#Handling null values in categorical columns
data = data.dropna()

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


###Data Visualization :

## Feature Encoding and Normalization

Before training the model, we should perform one-hot encoding for all categorical/discrete variables, normalize continuous variables and then combine all data to form the training set.

In [74]:
# create another copy of dataset and append encoded features to it
data_train = data.copy()
data_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [75]:
# select categorical features
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [76]:
#Encoding binary variables
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [77]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)
final_data.shape

(4261, 25)

In [78]:
final_data.columns

Index(['job_years', 'is_manager', 'hours_per_week',
       'telecommute_days_per_week', 'certifications', 'salary',
       'employment_status_Independent or freelancer or company owner',
       'employment_status_Part time', 'job_title_Analytics consultant',
       'job_title_Architect', 'job_title_DBA', 'job_title_Data Scientist',
       'job_title_Developer', 'job_title_Engineer', 'job_title_Manager',
       'job_title_Other', 'job_title_Principal database engineer',
       'job_title_Sales', 'job_title_Sr Consultant ',
       'education_Bachelors (4 years)', 'education_Doctorate/PhD',
       'education_Masters', 'education_None (no degree completed)',
       'is_education_computer_related_Unknown',
       'is_education_computer_related_Yes'],
      dtype='object')

In [79]:
final_data

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


## Train Test Split

In [80]:
y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


## Pre-processing data

In [81]:
# select numerical features
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols


['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [82]:
# Apply standard scaling on numeric data
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [83]:
X_train

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3267,0.081809,0,0.146913,-0.626010,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1828,1.259478,0,-0.483078,-0.626010,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3462,0.261102,1,-0.483078,-0.626010,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4243,-0.386616,1,-0.440931,-0.626010,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1738,1.259478,0,-0.483078,-0.019402,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.220030,1,-0.089593,2.407034,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2581,0.147268,0,-0.052438,-0.019402,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
198,0.259717,1,-0.269102,-0.626010,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
790,0.760290,0,-0.483078,-0.626010,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [84]:

reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [85]:
reg.coef_

array([ 1.83206267e+02,  3.39975734e+01,  1.96917184e+02,  2.25393623e+02,
        5.99648427e+01, -8.25661760e+01,  1.93441185e+02, -2.22797494e+03,
        1.11218669e+03,  8.29361433e+02,  4.88993790e+02,  5.14750674e+02,
        9.28640479e+02,  9.00403344e+02,  8.98238207e+01,  9.53093058e+02,
        6.84300858e+02, -1.79056769e-12,  3.04443540e+02,  3.19337106e+00,
        1.45946007e+02,  1.88378099e+02, -3.37985643e+02, -1.92031257e+02])

In [86]:
reg.intercept_

np.float64(6228.919357671032)

In [87]:
# Normalized MSE (Dividing by mean)
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

np.float64(212.47966654759458)

In [88]:
# Predict on the test data
y_pred = reg.predict(X_test)

In [89]:
#Evaluate the model on test data
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)


Mean Squared Error: 11872.61299649446


*A lower MSE indicates that the model's predictions are closer to the actual values on average, while a higher MSE suggests larger errors between predictions and actual values.
❗Pre-processing on Test data not done.

In [90]:
#Q1. Preprocess Test data and get predictions

In [91]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [92]:
X_test

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3400,0.261537,0,-0.510460,-0.627379,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1502,0.261537,0,2.440335,-0.627379,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3082,-0.172089,0,-0.505120,-0.627379,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1728,0.261537,1,-0.510460,0.582009,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
4178,0.261537,0,-0.510460,-0.627379,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,-0.007014,0,-0.286636,0.582009,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3860,-0.386089,0,-0.254200,-0.627379,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1781,-0.278500,0,-0.339715,-0.022685,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
886,0.261537,1,-0.510460,1.186704,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [93]:
y_hat = reg.predict(X_test)

In [94]:
#Predicted Values
y_hat

array([6976.68277343, 7471.75540216, 6571.24631251, 6766.6030395 ,
       6976.68277343, 7385.28491928, 7137.97273509, 6834.37445233,
       6697.55678167, 7419.50891787, 6379.09815821, 6774.62778996,
       6747.09063564, 7808.71969834, 7323.44463536, 6631.09425801,
       6401.37513476, 7481.74501384, 7323.44463536, 6876.51022291,
       6898.11867341, 7165.33201147, 7218.20799602, 7387.97882288,
       6507.4203506 , 7278.64947251, 8129.77695435, 7304.11049927,
       6134.61946129, 7820.77976793, 7971.53866101, 6499.0455744 ,
       7029.11846742, 7122.59247251, 7137.97273509, 6753.67375909,
       8582.15473092, 7883.29481869, 7240.49120704, 6757.52048602,
       7210.68362867, 5945.97022326, 7142.75815475, 6675.62125285,
       8521.21523259, 6908.51849345, 7159.22823157, 6876.97933386,
       7369.09570728, 7311.95892567, 6095.0435226 , 7870.87050731,
       7637.47607025, 6948.42789137, 7303.09902987, 6879.66727468,
       7132.2456876 , 7641.26785825, 6522.48499046, 7399.28014

In [95]:
# Normalized Mean Absolute Error (Dividing by mean)
mae_test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Abosulte Error: ",mae_test)


Test Mean Abosulte Error:  0.11538462611265854


In [96]:
# Normalized Mean Squared Error (Dividing by mean)
mse_test = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Squared Error: ",mse_test)


Test Mean Squared Error:  198.42350412287487
